
Καμάρης Άγγελος
sdi1900070

**Τεχνητή Νοημοσύνη ΙΙ**

(ακαδ. έτος 2022-23)

Άσκηση 2+bonus:


Άσκηση 2:

In [1]:
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
import matplotlib.pyplot as plt
!pip install transformers
import transformers
from google.colab import drive
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adamax
from transformers import BertModel, BertPreTrainedModel, BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
!pip install fuzzywuzzy
import string
from fuzzywuzzy import fuzz
!pip install sparqlwrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import time
from urllib.error import HTTPError
import torch.nn.functional as F

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 KB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 4.6 MB/s eta 0:00:00


In [2]:
drive.mount('/content/drive')
traindf=pd.read_csv('/content/drive/My Drive/my_train.csv')
testdf=pd.read_csv('/content/drive/My Drive/my_test.csv')
valdf=pd.read_csv('/content/drive/My Drive/my_val.csv')

Mounted at /content/drive


In [8]:
traindf=traindf.iloc[:6000, :]
testdf=testdf.iloc[:750, :]
valdf=valdf.iloc[:1000, :]

In [4]:
def keep_first_entity_span(spans):
  i=0
  newf=[]
  resf=0
  while i<len(spans):
    if spans[i]=='1':
      if i==0:
        newf.append(1)
        resf=i
      elif spans[i-1]=='0':
        newf.append(1)
        resf=i
      else:
        newf.append(0)
    else:
      newf.append(0)
    i+=1
  return resf


def keep_last_entity_span(spans):
  i=0
  newl=[]
  resl=0
  while i<len(spans):
    if spans[i]=='1':
      if i+1==len(spans):
         newl.append(1)
         resl=i
      elif spans[i+1]=='0':
        newl.append(1)
        resl=i
      else:
        newl.append(0)
    else:
      newl.append(0)
    i+=1
  return resl

def questionl(spans):
  x=len(spans)-1
  return x;

entity_starts=[]
entity_ends=[]
questionlen=[]
all=list(traindf['span'])
for tempspan in all:
  tempspan=tempspan[1:-1]
  f=keep_first_entity_span(tempspan.split(", "))
  l=keep_last_entity_span(tempspan.split(", "))
  s=questionl(tempspan.split(", "))
  entity_starts.append(f)
  entity_ends.append(l)
  questionlen.append(s)

tentity_starts=[]
tentity_ends=[]
tquestionlen=[]
all=list(testdf['span'])
for tempspan in all:
  tempspan=tempspan[1:-1]
  f=keep_first_entity_span(tempspan.split(", "))
  l=keep_last_entity_span(tempspan.split(", "))
  s=questionl(tempspan.split(", "))
  tentity_starts.append(f)
  tentity_ends.append(l)
  tquestionlen.append(s)


In [5]:
# Define your relation vocabulary
temp=list(traindf['relation_id'])
temp2=list(testdf['relation_id'])
temp.extend(temp2)
relation_vocab=list(set(temp))

# Define your dataset class
class myDataset(Dataset):
    def __init__(self, questions, relation_ids, entity_starts, entity_ends, relation_vocab, question_len):
        self.questions = questions
        self.relation_ids = relation_ids
        self.relation_vocab = relation_vocab
        self.entity_starts = entity_starts
        self.entity_ends = entity_ends
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.question_len = question_len
        
    def __len__(self):
        return len(self.questions)
    
    def __getitem__(self, idx):
        question = self.questions[idx]
        relation_id = self.relation_ids[idx]
        entity_start = self.entity_starts[idx]
        entity_end = self.entity_ends[idx]
        encoded = self.tokenizer.encode_plus(question, add_special_tokens=True, return_attention_mask=True, padding='max_length', max_length=30)
        input_ids = encoded['input_ids']
        attention_mask = encoded['attention_mask']
        label_id = self.relation_vocab.index(relation_id)
        question_len = self.question_len[idx]
        return {'input_ids': torch.tensor(input_ids), 'attention_mask': torch.tensor(attention_mask), 'labels': torch.tensor(label_id), 'entity_start': torch.tensor(entity_start), 'entity_end': torch.tensor(entity_end), 'question_len':torch.tensor(question_len)}

# Define custom BERT model with three heads
class QuestionAnsweringModel(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.relation_classifier = nn.Linear(config.hidden_size, len(relation_vocab))
        self.entity_start_classifier = nn.Linear(config.hidden_size, 1)
        self.entity_end_classifier = nn.Linear(config.hidden_size, 1)
        self.dropout = nn.Dropout(0.1)
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None,
            head_mask=None, inputs_embeds=None, labels=None, entity_start=None, entity_end=None,
            question_len=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,
                            position_ids=position_ids, head_mask=head_mask, inputs_embeds=inputs_embeds)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        relation_logits = self.relation_classifier(pooled_output)
        entity_start_logits = self.entity_start_classifier(outputs[0])
        entity_end_logits = self.entity_end_classifier(outputs[0])

        
        # set a mask for the entity_end logits
        mask = torch.arange(entity_end_logits.size(-1)).unsqueeze(0).unsqueeze(1).to(entity_end_logits.device)
        mask = mask >= question_len.unsqueeze(-1).unsqueeze(-1)
        entity_end_logits = entity_end_logits.masked_fill(mask, float('-inf'))

        return relation_logits, entity_start_logits, entity_end_logits

In [6]:
# Load your dataset and create dataloaders
questions = traindf['question']
relation_ids = traindf['relation_id']
entity_start_spans = entity_starts
entity_end_spans = entity_ends

dataset = myDataset(questions, relation_ids, entity_start_spans, entity_end_spans, relation_vocab, questionlen)
dataloader = DataLoader(dataset, batch_size=20)

tquestions = testdf['question']
trelation_ids = testdf['relation_id']
tentity_start_spans = tentity_starts
tentity_end_spans = tentity_ends
tdataset = myDataset(tquestions, trelation_ids, tentity_start_spans, tentity_end_spans, relation_vocab, tquestionlen)
tdataloader = DataLoader(tdataset, batch_size=20)

# Load pre-trained BERT model and modify output layer for relation classification
model = QuestionAnsweringModel.from_pretrained('bert-base-uncased', num_labels=len(relation_vocab))

# Define optimizer and loss function
optimizer = Adamax(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

# Fine-tune the model
count=0
model.train()
for epoch in range(2):
    running_loss = 0.0
    for batch in dataloader:
        count+=1
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        entity_start_labels = batch['entity_start']
        entity_end_labels = batch['entity_end']
        questionl = batch['question_len']
        # print(type(questionl))
        # print(questionl)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, entity_start=entity_start_labels, entity_end=entity_end_labels, question_len = questionl)
        loss = criterion(outputs[0].view(-1, len(relation_vocab)), labels.view(-1))+criterion(outputs[1].view(-1,30),entity_start_labels.view(-1))+criterion(outputs[2].view(-1,30),entity_end_labels.view(-1))
        running_loss += loss.item()/3
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(count)
    print(f"Epoch {epoch + 1}, loss = {running_loss / len(dataset)}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing QuestionAnsweringModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing QuestionAnsweringModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing QuestionAnsweringModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of QuestionAnsweringModel were not initialized from the model checkpoint at bert-base-uncased and are n

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [9]:
count=0
model.eval()
rela=0
relf=0
relp=0
relr=0
staa=0
staf=0
stap=0
star=0
enda=0
endf=0
endp=0
endr=0
start=0
end=0
for batch in dataloader:
    count+=1
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    entity_start_labels = batch['entity_start']
    entity_end_labels = batch['entity_end']
    questionl = batch['question_len']
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, entity_start=entity_start_labels, entity_end=entity_end_labels, question_len = questionl)
    predrel=torch.max(outputs[0], 1)[1]
    predsta=torch.max(outputs[1], 1)[1]
    predend=torch.max(outputs[2], 1)[1]

    rela+=accuracy_score(labels,predrel)
    relf+=f1_score(labels,predrel, average='macro', zero_division=0)
    relp+=precision_score(labels,predrel, average='macro', zero_division=0)
    relr+=recall_score(labels,predrel, average='macro', zero_division=0)
    staa+=accuracy_score(entity_start_labels,predsta)
    staf+=f1_score(entity_start_labels,predsta, average='macro', zero_division=0)
    stap+=precision_score(entity_start_labels,predsta, average='macro', zero_division=0)
    star+=recall_score(entity_start_labels,predsta, average='macro', zero_division=0)
    enda+=accuracy_score(entity_end_labels,predend)
    endf+=f1_score(entity_end_labels,predend, average='macro', zero_division=0)
    endp+=precision_score(entity_end_labels,predend, average='macro', zero_division=0)
    endr+=recall_score(entity_end_labels,predend, average='macro', zero_division=0)


print(f"Relation Accuracy {rela / count}, F1 score = {relf / count}, Precision score = {relp / count}, Recall score = {relr / count}")
print(f"Entity Start Accuracy {staa / count}, F1 score = {staf / count}, Precision score = {stap / count}, Recall score = {star / count}")
print(f"Entity End Accuracy {enda / count}, F1 score = {endf / count}, Precision score = {endp / count}, Recall score = {endr / count}")

Relation Accuracy 0.8916666666666652, F1 score = 0.775670338090493, Precision score = 0.772235982348882, Recall score = 0.7884071826572405
Entity Start Accuracy 0.9354999999999986, F1 score = 0.8766947221519592, Precision score = 0.8819703983332956, Recall score = 0.8869151599612911
Entity End Accuracy 0.9351666666666654, F1 score = 0.8664057952359282, Precision score = 0.8708497923299506, Recall score = 0.8765658582709184


Bonus:

In [10]:
# Step 1: Create a tokenizer instance
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Step 2: Encode the question
print("Please Provide a  question:")
question = input()
ql = torch.tensor(len(question.split()))
encoded_question = tokenizer(question, add_special_tokens=True, return_tensors='pt')


# Step 3: Pass the encoded question to the model
model.eval()
with torch.no_grad():
    outputs = model(
        input_ids=encoded_question['input_ids'],
        attention_mask=encoded_question['attention_mask'],
        question_len=ql
    )

# Step 4: Extract predicted relation, entity start, and entity end
_, predicted = torch.max(outputs[0], 1)
predicted = relation_vocab[predicted]

_, predicted_entity_start = torch.max(outputs[1], 1)
predicted_entity_start = predicted_entity_start.item()


_, predicted_entity_end = torch.max(outputs[2], 1)
predicted_entity_end = predicted_entity_end.item()



i=0
label=""
qlist=question.split()
for i in range(len(qlist)):
  if i>=predicted_entity_start and i<=predicted_entity_end:
    label+=qlist[i]
  if i<predicted_entity_end and i>=predicted_entity_start:
    label+=" "
  i+=1


# Remove punctuation from the label
label = label.translate(str.maketrans('', '', string.punctuation))

# List of labels to compare
labellist = list(set(list(traindf['entity_label']) + list(testdf['entity_label'])))

# Find the most similar label using fuzzywuzzy
most_similar_label, similarity_score = '', -1
for l in labellist:
    l = l.translate(str.maketrans('', '', string.punctuation))
    score = fuzz.token_set_ratio(l, label)
    if score > similarity_score:
        similarity_score = score
        most_similar_label = l


# Get the index and id of the most similar label
if most_similar_label in list(traindf['entity_label']):
    index = list(traindf['entity_label']).index(most_similar_label)
    id = list(traindf['entity_id'])[index]
else:
    index = list(testdf['entity_label']).index(most_similar_label)
    id = list(testdf['entity_id'])[index]


rid=predicted



sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)

def query_sparql(query):
    try:
        sparql.setQuery(query)
        results = sparql.query().convert()
        return results
    except HTTPError as e:
        if e.code == 429: # HTTP 429 = Too Many Requests
            print("Too many requests. Sleeping for 10 seconds...")
            time.sleep(10)
            return query_sparql(query)
        else:
            raise e



# Define the SPARQL query
query = """
SELECT ?item ?itemLabel 
WHERE 
{
  wd:"""+id+""" wdt:"""+rid+""" ?item.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

# Set the endpoint URL for the Wikidata Query Service API
url = 'https://query.wikidata.org/sparql'

# Set the headers to specify the format of the response
headers = {
    'Accept': 'application/sparql-results+json'
}

# Make the API call
results = query_sparql(query)

# Print the results
if(results['results']['bindings']!=[]):
  res='The answer found was: '+results['results']['bindings'][0]['itemLabel']['value']
else:
  res="Model could not find an answer"


print(res)

Please Provide a  question:
Who is an actor from Detroit?
Model could not find an answer
